In [ ]:
# use command line to make sure services are started
!brew services start mosquitto

In [ ]:
# use command line to make sure services are stopped
!brew services stop mosquitto

In [ ]:
# get ip address of laptop
!ifconfig |grep "inet "

In [ ]:
import paho.mqtt.client as mqtt
import numpy as np


# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, rc):
    print("Connected with result code "+str(rc))
    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("$ubicomp/#")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    
    if '$ubicomp/' not in msg.topic:
        return 
    
    team = msg.topic[9:]
    msg = msg.payload
    
    print('Received', msg, 'from team',team)
    
    # now let's send something that this team might be 
    # subscribed to (randomly)
    if np.random.rand() > 0.5:
        # now send something useless!
        client.publish(topic='$SYS/Eric',payload='Hello There!')
    

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("localhost", 1883, 60)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
client.loop_forever()

In [ ]:
import paho.mqtt.client as mqtt
from ringbuffer import RingBuffer
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation
from matplotlib.lines import Line2D
%matplotlib notebook

BUFFER_SIZE = 10
data = {} 

# setup the figure to plot with
fig, ax1 = plt.subplots(1,1)
# setup the plot
ax1.set_xlim(0, BUFFER_SIZE)
ax1.set_ylim(0, 4096)

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, rc):
    print("Connected with result code "+str(rc))
    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("$ubicomp/#")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    
    if '$ubicomp/' not in msg.topic:
        return 
    
    team = msg.topic[9:]
    payload = msg.payload
    
    # now let's send something that this team might be 
    # subscribed to (randomly)
    if np.random.rand() > 0.5:
        val = 'OFF' if np.random.rand() > 0.5 else 'ON'
        client.publish(topic='$'+team,payload='LED '+val)
    
    if team not in data:
        data[team] = {}
        data[team]['buffer'] = RingBuffer(BUFFER_SIZE)
        data[team]['line'], = plt.plot(data[team]['buffer'].get_samples,label=team)
    
    data[team]['buffer'].insert_new(np.array(int(payload)))
    line1 = data[team]['line']
    ydata = data[team]['buffer'].get_samples
    line1.set_xdata(np.linspace(1,BUFFER_SIZE-1,BUFFER_SIZE))
    line1.set_ydata(ydata)  # set the data
    plt.draw()  # and draw it out
    plt.legend()
    fig.canvas.draw()  # pause so that the drawing updates 

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("localhost", 1883, 60)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
client.loop_forever()

In [ ]:
client.disconnect()